In [ ]:
import boto3
from datetime import datetime, timezone, timedelta


session = boto3.session.Session()
s3_client = session.client('s3')
s3 = boto3.resource('s3')

In [ ]:
#define how many days to look back
days_to_go_back = 3

In [ ]:
stamp = datetime.now(timezone.utc) + timedelta(days= -days_to_go_back)

new = {}
response = s3_client.list_buckets()
for b in response['Buckets']:
    bucket = b['Name']
    if 'submissions-czi' in bucket:
        files = {}
        my_bucket = s3.Bucket(bucket)
        for my_bucket_object in my_bucket.objects.all():
            file_name = my_bucket_object.key
            if my_bucket_object.last_modified > stamp and file_name.endswith('/') != True:
                date = str(my_bucket_object.last_modified.date())
                if date in files:
                    files[date].append(file_name)
                else:
                    files[date] = [file_name]
        if files:
            new[bucket] = files

for k,v in new.items():
    allfiles = [f for l in v.values() for f in l]
    print(f'{k}: {str(len(allfiles))} new files')

In [ ]:
#select a bucket to investigate specific files
bucket = 'submissions-czi011eye'

for k,v in new[bucket].items():
    print(k)
    print('\n'.join(v) + '\n')